In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from utils.training import numpy_metrics

In [3]:
def softmax(x, axis=1):
    return np.exp(x) / np.sum(np.exp(x),axis=axis, keepdims=True)

In [4]:
def ece(outputs, labels, name="", bins=np.arange(0.1, 1.05, 0.05), print_name=False, background_label = 11):

    if outputs is None:
        ece = None
    else:
        confidences = np.max(outputs, 1)
        predictions = np.argmax(outputs,1)
        bin_lowers = bins[:-1]
        bin_uppers = bins[1:]
        
        accuracies = predictions == labels 

        #ece = Variable(torch.zeros(1)).type_as(confidences)
        ece = 0.0
        for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
            # Calculated |confidence - accuracy| in each bin
            in_bin = (confidences > bin_lower) * (confidences < bin_upper)
            prop_in_bin = in_bin.mean()
            if prop_in_bin > 0:
                accuracy_in_bin = accuracies[in_bin].mean()
                avg_confidence_in_bin = confidences[in_bin].mean()
                ece += np.abs(avg_confidence_in_bin-accuracy_in_bin) * prop_in_bin
    if print_name:
        print(name, end=": ")
    if ece is not None:
        print("%.3f"%ece)
    else:
        print("-")
    return ece



In [5]:
def compute_mIOU_acc(loc, output_probs = True, background_class = 11):
    npz_arr = np.load(loc)
    
    if output_probs:
        pred_probs = npz_arr['predictions']
    else:
        pred_probs = softmax(npz_arr['preds'],1)
    targets = npz_arr['targets']
    
    I, U, acc = numpy_metrics(pred_probs, targets)
    mIOU = np.mean(I / U)
    
    filtered_targets = targets.reshape(-1)
    background_labels = filtered_targets == background_class
    filtered_targets = filtered_targets[~background_labels]
    
    filtered_pred_probs = pred_probs.reshape(-1, pred_probs.shape[1])
    print(filtered_pred_probs.shape, filtered_pred_probs.sum(1).mean())
    filtered_pred_probs = filtered_pred_probs[~background_labels, :]
    
    ece_val = ece(filtered_pred_probs, filtered_targets, print_name=False)
    print(acc, mIOU, ece_val, loc)
    return acc, mIOU, ece_val

In [8]:
compute_mIOU_acc('/home/wesley/Desktop/nfs01_tesla/swa_uncertainties/segmentation/test_preds/dropout_new/preds.npz')

(40262400, 11) 1.000000007170505
0.089
0.900859621513635 0.6231925359426722 0.08924742584234689 /home/wesley/Desktop/nfs01_tesla/swa_uncertainties/segmentation/test_preds/dropout_new/preds.npz


(0.900859621513635, 0.6231925359426722, 0.08924742584234689)

In [6]:
compute_mIOU_acc('/home/wesley/Desktop/nfs01_tesla/swa_uncertainties/segmentation/test_preds/swag_new/preds.npz')

(40262400, 11) 1.000000014855937
0.091
0.9101354251839606 0.63325866789444 0.09144399240896578 /home/wesley/Desktop/nfs01_tesla/swa_uncertainties/segmentation/test_preds/swag_new/preds.npz


(0.9101354251839606, 0.63325866789444, 0.09144399240896578)

In [8]:
compute_mIOU_acc('/home/wesley/Desktop/tesla/swa_uncertainties/experiments/segmentation/swag_980.npz', 
                 output_probs = False)

(40262400, 11) 0.9999993
0.092
0.9088837405143834 0.6325884654568185 0.09178933595602778 /home/wesley/Desktop/tesla/swa_uncertainties/experiments/segmentation/swag_980.npz


(0.9088837405143834, 0.6325884654568185, 0.09178933595602778)

In [9]:
compute_mIOU_acc('/home/wesley/Desktop/tesla/swa_uncertainties/experiments/segmentation/sgd_850_final.npz', 
                 output_probs = False)

(40262400, 11) 0.99999833
0.091
0.9105683390670791 0.6459073234037088 0.0909109173528229 /home/wesley/Desktop/tesla/swa_uncertainties/experiments/segmentation/sgd_850_final.npz


(0.9105683390670791, 0.6459073234037088, 0.0909109173528229)